In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/scratch/cvlab/home/gwizdala/code/pidnet")

In [10]:
import numpy as np
import torch

from functools import reduce
from pathlib import Path

from powerlines.utils import load_yaml

folds = load_yaml(Path("configs/powerlines/folds.yaml"))
timestamps = set(reduce(lambda a, b: a + b, folds.values(), []))

cables_distance_masks_folder = Path(
    "/scratch/cvlab/home/gwizdala/dataset/processed/daedalean/complete_frames/distance_masks/visible"
)
poles_distance_masks_folder = Path(
    "/scratch/cvlab/home/gwizdala/dataset/processed/daedalean/complete_frames/poles_distance_masks"
)

cable_filepaths = list(filter(lambda path: int(path.stem) in timestamps, cables_distance_masks_folder.glob("*.npy")))
poles_filepaths = list(filter(lambda path: int(path.stem) in timestamps, poles_distance_masks_folder.glob("*.npy")))
assert len(cable_filepaths) == len(poles_filepaths)

In [13]:
from tqdm import tqdm

from powerlines.data.utils import downsample_labels

frame_size = 187 * 256  # downsampled 16x shape

In [15]:
from powerlines.data.utils import nevbw_labels_from_distance_mask

# Cable frequency
total_pos_count = 0
for max_cells_away in [1, 4, 8]:
    for filepath in tqdm(cable_filepaths):
        cables_distance_mask = np.load(str(filepath))
        cables_label = nevbw_labels_from_distance_mask(cables_distance_mask, max_cells_away=max_cells_away)
        downsampled_label = downsample_labels(
            torch.from_numpy(cables_label).float(), grid_size=16, adjust_to_divisible=False
        ).long().detach().cpu().numpy()

        pos_count = np.count_nonzero(downsampled_label == 1)
        total_pos_count += pos_count

    pos_frequency = total_pos_count / (len(cable_filepaths) * frame_size)
    neg_frequency = 1 - pos_frequency

    print(f"--- Max cells away: {max_cells_away} ---")
    print(f"Cables frequency: {pos_frequency}")
    print(f"Cables weight: {1 / pos_frequency}")
    print(f"Background weight: {1 / neg_frequency}")

100%|██████████| 865/865 [03:48<00:00,  3.78it/s]


--- Max cells away: 1 ---
Cables frequency: 0.011903128960464901
Cables weight: 84.01152363562589
Background weight: 1.0120465202444595


100%|██████████| 865/865 [03:46<00:00,  3.82it/s]


--- Max cells away: 4 ---
Cables frequency: 0.04162001850792866
Cables weight: 24.026899454874076
Background weight: 1.0434274706397058


100%|██████████| 865/865 [03:39<00:00,  3.94it/s]

--- Max cells away: 8 ---
Cables frequency: 0.09412081060090878
Cables weight: 10.624642877760602
Background weight: 1.1038999589595861


In [16]:
# Poles frequency
total_pos_count = 0
for max_cells_away in [1, 4, 8]:
    for filepath in tqdm(poles_filepaths):
        poles_distance_mask = np.load(filepath)
        poles_label = nevbw_labels_from_distance_mask(poles_distance_mask, max_cells_away=max_cells_away)
        downsampled_poles_label = downsample_labels(
            torch.from_numpy(poles_label).float(), grid_size=16, adjust_to_divisible=False
        ).long().detach().cpu().numpy()

        pos_count = np.count_nonzero(downsampled_poles_label == 1)
        total_pos_count += pos_count

    pos_frequency = total_pos_count / (len(poles_filepaths) * frame_size)
    neg_frequency = 1 - pos_frequency

    print(f"--- Max cells away: {max_cells_away}")
    print(f"Poles frequency: {pos_frequency}")
    print(f"Poles weight: {1 / pos_frequency}")
    print(f"Background weight: {1 / neg_frequency}")

100%|██████████| 865/865 [04:04<00:00,  3.54it/s]


--- Max cells away: 1
Poles frequency: 0.0059538828011498875
Poles weight: 167.95762251263454
Background weight: 1.0059895438432247


100%|██████████| 865/865 [03:41<00:00,  3.91it/s]


--- Max cells away: 4
Poles frequency: 0.028293150714042842
Poles weight: 35.34424320949403
Background weight: 1.029116961288102


100%|██████████| 865/865 [03:39<00:00,  3.94it/s]

--- Max cells away: 8
Poles frequency: 0.07984698115977867
Poles weight: 12.52395501338916
Background weight: 1.0867757639489346
